In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras import regularizers
from keras.callbacks import TensorBoard
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import load_model

from keras.layers import Input
from keras.models import Model

from time import time

from preprocess import fetch_data

In [3]:
from sklearn.model_selection import train_test_split

## Format Data

In [9]:
X, y = fetch_data(['data/200_fullstate_1v0.log'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


AssertionError: 

In [8]:
print(X_train.shape)
print(y_train.shape)

(14407, 68)
(14407, 10)


## Make Keras Model

In [ ]:

main_input = Input(shape=(68,), name='main_input')

hidden_layer_1 = Dense(1000, activation='relu')(main_input)
hidden_layer_2 = Dense(512, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(200, activation='relu')(hidden_layer_2)
hidden_layer_4 = Dense(64, activation='relu')(hidden_layer_3)


action_prob_output = Dense(4, activation='softmax', name='action_probabilities')(hidden_layer_4)
action_prob_output = Dense(6, name='action_params')(hidden_layer_4)

model.compile(optimizer='adam', 
              loss={'action_probabilities': 'categorical_crossentropy', 'action_params': 'mean_squared_error'})

# Printing a summary of the layers and weights in your model
model.summary()

In [ ]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

## Fit Model

In [ ]:


# And trained it via:
model.fit({'main_input': X_train},
          {'action_probabilities': y_train[:4], 'action_params': y_train[4:]},
          epochs=15, batch_size=32, validation_data=(X_test, y_test))


## Save Model

In [ ]:
model.save('models/basic_model.h5')  # creates a HDF5 file 'my_model.h5'

## Results

In [ ]:
score = model.evaluate(X_train, y_train, verbose=0)
print('Training score:', score[0])
print('Training accuracy:', score[1])

score = model.evaluate(X_test, y_test, verbose=0)
print('Testing score:', score[0])
print('Testing accuracy:', score[1])